In [3]:
# import packages
import torch
import torch.nn as nn
from torch import optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
# print(torch.cuda.is_available())

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [14]:
# import the time series
ts=pd.read_csv("dataset.csv")

# create the timestamp for the date
ts['datetime'] = pd.to_datetime(ts['datetime'],format="%Y-%m-%dT%H:%M:%S")
ts = ts.set_index(pd.DatetimeIndex(ts['datetime']))
ts.drop('datetime', axis=1)

# Aggregate to hour
ts = ts.resample('H').sum()

# calculate the return and make the time series stationary
# ts['diff']=np.log(ts['values']) - np.log(ts['values'].iloc[0])

# Split data into train and test set
train_size = int(len(ts) * 0.8)
X_train, X_test = ts[0:train_size], ts[train_size:len(ts)]

In [17]:
# make training and test sets in torch
X_train = torch.Tensor(X_train.values)

X_test = torch.Tensor(X_test.values)

X_train = X_train.view([len(X_train), -1, 1])
X_test = X_test.view([len(X_test), -1, 1])

In [30]:
test_size = 0.2
# num_train = int((1-test_size) * num_datapoints)

# Network params
input_size = 8128
# If `per_element` is True, then LSTM reads in one timestep at a time.
per_element = True
if per_element:
    lstm_input_size = 1
else:
    lstm_input_size = len(X_train)
# size of hidden layers
h1 = 32
output_dim = 1
num_layers = 2
learning_rate = 1e-3
num_epochs = 500
dtype = torch.float

In [35]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)
    
model = LSTM(lstm_input_size, h1, batch_size=100, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

/home/david/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [37]:
hist = np.zeros(num_epochs)

In [36]:
for epoch in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

RuntimeError: invalid argument 2: size '[4064 x 100 x -1]' is invalid for input with 4064 elements at /opt/conda/conda-bld/pytorch_1533672544752/work/aten/src/TH/THStorage.cpp:80

In [ ]:
plt.plot(y_pred.detach().numpy(), label="Preds")
plt.plot(y_train.detach().numpy(), label="Data")
plt.legend()
plt.show()

plt.plot(hist, label="Training loss")
plt.legend()
plt.show()